<a href="https://colab.research.google.com/github/jsansao/s3nr/blob/main/GeracaoMascarasVozSintetica_UNET.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Segmentação de imagens espectrográficas com UNET

## Geração do dataset de treinamento

### Carregando bibliotecas para geração dos espectrogramas e máscaras

In [2]:
pip install librosa matplotlib


In [3]:
import librosa
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

### Carregando o dataset de treinamento

In [4]:
!git clone https://github.com/jsansao/synthvoice_unet.git

Cloning into 'synthvoice_unet'...
remote: Enumerating objects: 5241, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 5241 (delta 0), reused 3 (delta 0), pack-reused 5238
Receiving objects: 100% (5241/5241), 372.10 MiB | 29.79 MiB/s, done.
Resolving deltas: 100% (127/127), done.
Updating files: 100% (6423/6423), done.


In [5]:
def calcular_espectrograma(file_path):
    # Carregar o arquivo WAV
    audio_signal, sample_rate = librosa.load(file_path)

    # Calcular o espectrograma usando a STFT
    n_fft = 1024
    hop_length = int(n_fft * 0.1)  # Overlap de 90%

    array_length = len(audio_signal)
    #trecho = array_length // 8;  # corta pedaço do sinal

    trecho = array_length ;  # corta pedaço do sinal

    stft = librosa.stft(audio_signal[:trecho], n_fft=n_fft, hop_length=hop_length)

    # Converter o espectrograma para escala de decibéis
    #spectrogram = librosa.amplitude_to_db(np.abs(stft))

    # Calculate the number of elements to remove (20% from start and end)
    remove_percentage = 0.2

    num_elements_to_remove = int(len(stft) * remove_percentage)

    # Slice the array to remove elements (20% from start and end)
    trimmed_stft = stft[:, num_elements_to_remove:-num_elements_to_remove]
    spectrogram = np.abs(trimmed_stft)

    return spectrogram

In [6]:
def calcular_espectrograma_mascara(diretorio, diretorio_semruido, diretorio_saida, diretorio_mascara, limiar):

  # Listar todos os arquivos no diretório
  arquivos_wav = [f for f in os.listdir(diretorio) if f.endswith('.wav')]

  #espectrogramas_array = []

  # Iterar sobre cada arquivo e calcular o espectrograma
  for arquivo in arquivos_wav[:]:

    caminho_arquivo = os.path.join(diretorio, arquivo)
    caminho_arquivo_semruido = os.path.join(diretorio_semruido, arquivo)

    espectrograma = calcular_espectrograma(caminho_arquivo)
    espectrograma_semruido = calcular_espectrograma(caminho_arquivo)

    mascara = np.zeros_like(espectrograma_semruido)
    mascara[espectrograma_semruido >= limiar] = 255

    #img_espectrograma = Image.fromarray(librosa.amplitude_to_db(espectrograma))
    espectrograma_int = (espectrograma * 255).astype(np.uint8)
    img_espectrograma = Image.fromarray((espectrograma)).convert('RGB')
    img_mascara = Image.fromarray(mascara.astype(np.uint8))

    arquivo_semextensao, _ = os.path.splitext(arquivo)

    arquivo_saida = arquivo_semextensao + '.png'

    caminho_arquivo_saida = os.path.join(diretorio_saida, arquivo_saida)
    caminho_mascara = os.path.join(diretorio_mascara, arquivo_saida)


    img_espectrograma.save(caminho_arquivo_saida)
    img_mascara.save(caminho_mascara)




In [7]:
def calcular_espectrograma_npy_mascara(diretorio, diretorio_semruido, diretorio_saida, diretorio_mascara, limiar):

  # Listar todos os arquivos no diretório
  arquivos_wav = [f for f in os.listdir(diretorio) if f.endswith('.wav')]

  #espectrogramas_array = []

  # Iterar sobre cada arquivo e calcular o espectrograma
  for arquivo in arquivos_wav[:]:

    caminho_arquivo = os.path.join(diretorio, arquivo)
    caminho_arquivo_semruido = os.path.join(diretorio_semruido, arquivo)

    espectrograma = calcular_espectrograma(caminho_arquivo)
    espectrograma_semruido = calcular_espectrograma(caminho_arquivo)

    mascara = np.zeros_like(espectrograma_semruido)
    mascara[espectrograma_semruido >= limiar] = 255

    arquivo_semextensao, _ = os.path.splitext(arquivo)

    arquivo_saida = arquivo_semextensao + '.npy'


    caminho_arquivo_saida = os.path.join(diretorio_saida, arquivo_saida)
    caminho_mascara = os.path.join(diretorio_mascara, arquivo_saida)

    np.save(caminho_arquivo_saida, espectrograma)
    np.save(caminho_mascara, mascara)


### Geração de imagens

In [8]:
import os


!rm -fr train

diretorio = '/content/synthvoice_unet/amostras_jitter_SNR/'
diretorio_semruido = '/content/synthvoice_unet/amostras_jitter_SNR_semruido/'

diretorio_saida = '/content/train/imgs/'
diretorio_mascara = '/content/train/masks/'

if not os.path.exists(diretorio_saida):
        os.makedirs(diretorio_saida)

if not os.path.exists(diretorio_mascara):
        os.makedirs(diretorio_mascara)

limiar = 0.1


calcular_espectrograma_npy_mascara(diretorio, diretorio_semruido, diretorio_saida, diretorio_mascara, limiar)


## Treinamento UNET

In [9]:
!git clone https://github.com/jsansao/UNet-Pytorch-Customdataset.git

Cloning into 'UNet-Pytorch-Customdataset'...
remote: Enumerating objects: 62, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 62 (delta 21), reused 42 (delta 9), pack-reused 0
Receiving objects: 100% (62/62), 24.22 MiB | 25.80 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [10]:
!rm -rf /content/UNet-Pytorch-Customdataset/data/

In [11]:
cd /content/UNet-Pytorch-Customdataset/

/content/UNet-Pytorch-Customdataset


In [12]:
!mv /content/train/ /content/UNet-Pytorch-Customdataset/data/

In [13]:
!python train.py --epochs 10 --batch-size 32

INFO: Using device cuda
INFO: Creating dataset with 3211 examples
INFO: Scanning mask files to determine unique values
100% 3211/3211 [00:23<00:00, 137.27it/s]
INFO: Unique mask values: [0.0, 255.0]
Epoch 1/10:   1% 32/2890 [00:08<13:15,  3.59img/s]loss: 1.2614481449127197
Epoch 1/10:   2% 64/2890 [00:09<05:49,  8.09img/s]loss: 1.1269829273223877
Epoch 1/10:   3% 96/2890 [00:10<03:47, 12.30img/s]loss: 0.9820834994316101
Epoch 1/10:   4% 128/2890 [00:11<02:49, 16.27img/s]loss: 0.8435583114624023
Epoch 1/10:   6% 160/2890 [00:12<02:17, 19.80img/s]loss: 0.7319466471672058
Epoch 1/10:   7% 192/2890 [00:13<01:58, 22.78img/s]loss: 0.6448454856872559
Epoch 1/10:   8% 224/2890 [00:14<01:46, 25.12img/s]loss: 0.5465792417526245
Epoch 1/10:   9% 256/2890 [00:15<01:37, 26.98img/s]loss: 0.48335206508636475
Epoch 1/10:  10% 288/2890 [00:16<01:32, 28.21img/s]loss: 0.46453189849853516
Epoch 1/10:  11% 320/2890 [00:17<01:27, 29.37img/s]loss: 0.436676949262619
Epoch 1/10:  12% 352/2890 [00:18<01:24, 30.

In [15]:
!python predict.py --model ./checkpoints/checkpoint_epoch10.pth -i /content/UNet-Pytorch-Customdataset/data/imgs/SNR0218J000S000VaF220_1.npy --viz --output ./0_OUT.jpg

INFO: Loading model ./checkpoints/checkpoint_epoch10.pth
INFO: Using device cuda
INFO: Model loaded!
INFO: Predicting image /content/UNet-Pytorch-Customdataset/data/imgs/SNR0218J000S000VaF220_1.npy ...
INFO: Mask saved to ./0_OUT.jpg
INFO: Visualizing results for image /content/UNet-Pytorch-Customdataset/data/imgs/SNR0218J000S000VaF220_1.npy, close to continue...
Figure(640x480)
